In [ ]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
import re
import csv
import json

import os

In [19]:
with open([f for f in os.listdir(os.getcwd()) if f.endswith('json')][0]) as f:
    config = json.load(f)

api_key = config["YOUTUBE_API_KEY"]

In [7]:
youtube = build("youtube", "v3", developerKey=api_key)

user_name ="https://www.youtube.com/@MyNameIsDeya"

In [9]:
def get_channel_id_from_username(username):
    res = youtube.search().list(
        q=username,
        type='channel',
        part='snippet',
        maxResults=1
    ).execute()
    return res['items'][0]['snippet']['channelId']

def get_uploads_playlist_id(channel_id):
    res = youtube.channels().list(id=channel_id, part='contentDetails').execute()
    return res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_video_ids(playlist_id):
    video_ids = []
    next_page_token = None
    while True:
        res = youtube.playlistItems().list(
            playlistId=playlist_id,
            part='contentDetails',
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        video_ids += [item['contentDetails']['videoId'] for item in res['items']]
        next_page_token = res.get('nextPageToken')
        if next_page_token is None:
            break
    return video_ids

def get_video_details(video_ids):
    video_data = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(
            id=','.join(video_ids[i:i+50]),
            part='snippet,statistics'
        ).execute()
        for video in res['items']:
            data = {
                'video_id': video['id'],
                'title': video['snippet']['title'],
                'published_at': video['snippet']['publishedAt'],
                'views': int(video['statistics'].get('viewCount', 0)),
                'likes': int(video['statistics'].get('likeCount', 0)),
                'comments': int(video['statistics'].get('commentCount', 0)),
            }
            video_data.append(data)
    return pd.DataFrame(video_data)

In [ ]:
channel_id=get_channel_id_from_username(user_name)

playlist_id=get_uploads_playlist_id(channel_id)


In [21]:
video_ids=get_video_ids(playlist_id)

In [25]:
df=get_video_details(video_ids)

In [29]:
df.to_csv('youtube_data.csv',index=False)